# Import Libraries

In [19]:
import pandas as pd
import numpy as np
import pytz
from datetime import datetime
import datetime as dt
import glob
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.colors import LogNorm
from datetime import timedelta

# Define Paths

- You will need to change the paths here

In [60]:
# input_path is the path to the folder in which your shiny data is stored
# Update this with your desired path
input_path = '/Users/vineblankenship/Research/ASCENT/SMPS_Data_Analysis/JTcomparison/picoshiny/shiny_april24_to_aug14_2025/' 

# clean path: once you concatenate the data from the shiny csv files and remove any problematis dates, you save 
# this new df into a csv file in clean_path
# Update this with your desired path
clean_path = '/Users/vineblankenship/Research/ASCENT/PicoPaper/pico_april24_to_aug14_2025.csv'

# plot_ready_folder is the folder in which you store your final csv file after changing the timezine in the final cleaning step
# Update this with your desired path
plot_ready_folder = '/Users/vineblankenship/Research/ASCENT/PicoPaper/site_dfs/pico'  

# cleaned_csv is your final csv file after changing the timezine in the final cleaning step 
# Update this with your desired filename
cleaned_csv = 'pico_late_spring_to_mid_august_2025.csv'


# Concatenation code

## Concatenate the csv files (This code is meant for Shiny data)

- Load in Shiny data and concatenate it into one dataframe

In [61]:
csv_files = glob.glob(os.path.join(input_path, '*.csv'))

if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {input_path}")

def read_csv_after_scan_number(file):
    # Find the header row dynamically
    with open(file, 'r', encoding='utf-8', errors='ignore') as f:
        for i, line in enumerate(f):
            if "DateTime Sample Start" in line:
                header_row = i
                break
    
    # Now read the file starting from that row
    df = pd.read_csv(file, skiprows=header_row, low_memory=False).dropna(how='all')
    return df

# Apply to all your CSVs
df_list = (read_csv_after_scan_number(file) for file in csv_files)

# Concatenate all DataFrames
df = pd.concat(df_list, ignore_index=True).drop_duplicates().reset_index(drop=True)

# Parse datetime
df['DateTime Sample Start'] = pd.to_datetime(
    df['DateTime Sample Start'], format='%d/%m/%Y %H:%M:%S', errors='coerce'
)

# Filter out rows with errors/warnings
df = df[
    (df['Detector Status'] == 'Normal Scan') &
    (df['Classifier Errors'] == 'Normal Scan')
]

# Sort by time
df = df.sort_values('DateTime Sample Start').reset_index(drop=True)


In [62]:
df

,Scan Number,DateTime Sample Start,Test Name,Detector Status,Classifier Errors,Communication Status,Neutralizer Status,Detector Inlet Flow (L/min),Detector Counting Flow (L/min),Impactor Flow (L/min),...,572.55,593.52,615.27,637.80,661.17,685.39,710.50,736.53,763.51,791.48
0,574,2025-04-24 00:01:13,ASCENT_2023_LA_150s,Normal Scan,Normal Scan,0,ON,0.6,0.3,0.6,...,0.000,0.000,50.745,0.000,52.524,0.000,0.000,0.000,38.041,31.999
1,575,2025-04-24 00:03:45,ASCENT_2023_LA_150s,Normal Scan,Normal Scan,0,ON,0.6,0.3,0.6,...,49.424,0.000,51.130,0.000,0.000,53.857,0.000,48.486,0.000,0.000
2,576,2025-04-24 00:06:16,ASCENT_2023_LA_150s,Normal Scan,Normal Scan,0,ON,0.6,0.3,0.6,...,48.989,0.000,0.000,0.000,52.524,53.415,0.000,0.000,0.000,0.000
3,577,2025-04-24 00:08:47,ASCENT_2023_LA_150s,Normal Scan,Normal Scan,0,ON,0.6,0.3,0.6,...,49.424,0.000,0.000,0.000,0.000,53.415,54.301,0.000,0.000,0.000
4,578,2025-04-24 00:11:18,ASCENT_2023_LA_150s,Normal Scan,Normal Scan,0,ON,0.6,0.3,0.6,...,48.989,49.863,0.000,52.075,0.000,53.415,0.000,48.319,38.500,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61996,4673,2025-08-15 03:52:24,ASCENT_2023_LA_150s,Normal Scan,Normal Scan,0,ON,0.6,0.3,0.6,...,49.449,0.000,0.000,51.662,0.000,53.445,0.000,0.000,0.000,59.839
61997,4674,2025-08-15 03:54:55,ASCENT_2023_LA_150s,Normal Scan,Normal Scan,0,ON,0.6,0.3,0.6,...,0.000,0.000,101.543,0.000,52.553,0.000,54.328,0.000,0.000,26.347
61998,4675,2025-08-15 03:57:26,ASCENT_2023_LA_150s,Normal Scan,Normal Scan,0,ON,0.6,0.3,0.6,...,98.464,100.220,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
61999,4676,2025-08-15 03:59:58,ASCENT_2023_LA_150s,Normal Scan,Normal Scan,0,ON,0.6,0.3,0.6,...,0.000,49.890,0.000,0.000,0.000,53.883,57.489,0.000,0.000,0.000


# Cleaning Code

## Remove bad data: 

Examples:

- The instrument was stopped or disconnected from internet but the timestamps were filled with constant concentrations over the stop period

- Instrument error

- Outlier atmospheric conditions

In the below cell, I remove the dates affected by 4th of July fireworks

In [63]:
# Define the start and end times to remove
# Remember the time here is still in UTC
start_remove = pd.Timestamp('2025-07-04 08:00:00')
end_remove = pd.Timestamp('2025-07-06 08:00:00')

# Filter out the unwanted time range
df = df[~((df['DateTime Sample Start'] >= start_remove) & 
          (df['DateTime Sample Start'] <= end_remove))]  # You can do this however many times you need to!


## Save this concatenated CSV

In [ ]:
# Save the cleaned DataFrame

df.to_csv(clean_path, index=False)


## The following 2 cells ensure that the time gaps show up as white space on the time series plots by inserting rows for the missing time stamps filled with NaN values

- Load in clean SMPS data with loadSMPSdata

- The data you load in here is the data you saved as a csv in the previous cell

In [64]:
def loadSMPSdata( clean_path = clean_path,
        utc = True,
        timezone_in_raw_data = 'UTC',
        timezone_for_analysis = 'America/Los_Angeles',
        full_AIM_data = True,
        use_TSI_dNdlogD = True ):
    '''
    loaddata reads a csv file identified by data_path and creates a dataframe
    with a datetime index containing all of the data in the csv file.  It is
    assumed that the datetime index is UTC time,

    Parameters
    ----------
    data_path : TYPE, optional
        DESCRIPTION. The default is 
          '/Users/flagan/Documents/PythonCode/Aerosols/' + 
          'ASCENT_Data/Pico_Rivera/smps_apr_may_2024rxw.csv'.timezonestr: 
          TYPE, optional
    timezone_in_raw_data 
        DESCRIPTION
            The string representation of the time zone in which
          in which the raw data are supplied.  The default is 'UTC'
    timezone_for_analysis
        DESCRIPTION. The string representation of the time zone to
            be used in data analysis and plotting.  The default is 
            'America/Los_Angeles'.
    full_AIM_data
        DESCRIPTION Default = True -- corresponding to a dataset that includes
            the 52 lines of header data, and the empty (all zero value) size 
            distribution data columns.  When True, the header data are skipped,
            and the empty size distribution columns are deleted.
    Returns
    -------
    df : TYPE Pandas dataframe
        DESCRIPTION
            The output dataframe has a time-zone naive datetime index that
            has been converted to the time zone specified by 
            timezone_for_analysis from the time zone of the input data 
            specified by timezone_in_raw_data
    
    Richard C. Flagan
    16 Sept 2024
    --------------------------------
    Indices of key columns in the TSI AIM data set that includes raw concentrations
    0 = scan number
    1 = Datetime -- though this is not the original name
    42 = dNdlogDp (TSI) number reporting begins 
    184 = dNdlogDp (TSI) useful data reporting begins 12.98 nm
    414 = dNdlogDp (TSI) last useful data reporting 
    425 = dNdlogDp (TSI) last number report
    426 = Nraw reporting begins 12.98 nm  size format _12.98
    798 = Nraw last useful value 813.12 nm
    809 = Nraw reporting end
    '''
    
    # Simplified the following line to fit my path
    data_path  = clean_path
    if not full_AIM_data:
        dfraw = pd.read_csv(data_path)
    else:
        dfraw = pd.read_csv(data_path)  # ← no skiprows!
        dfraw.columns = dfraw.columns.str.strip()  # just in case of leading/trailing spaces
        dfraw.dropna(subset=['Scan Number'], inplace=True)

        if use_TSI_dNdlogD:
            dfraw.drop(dfraw.columns[414:], axis=1, inplace=True)
            
        else:
            pass
            
    # Changed Datetime to DateTime Sample Start in the following lines 
    dfraw['DateTime Sample Start'] = pd.to_datetime(dfraw['DateTime Sample Start'], format='%Y-%m-%d %H:%M:%S')
    dfraw.set_index('DateTime Sample Start', inplace=True)
    dfraw.index = dfraw.index.strftime('%Y-%m-%d %H:%M:%S')
    return dfraw

def print_index( df ):
    for i in range( len(df) ):
        print ( i, df.index[i] )



- This clean_data function below inserts rows for the missing time stamps and fills them with NaN values

- This makes the time gaps appear as white space in the time series colormap plots

In [65]:
def clean_data(df, vmin=1.e-2, excess_time_factor=1.1, scantime=150):
    '''
    clean_data addresses defects in the dataset. If the time between successive
    finite-duration measurements is too long (1.1 * scan_time), a blank (NaN) 
    measurement interval is inserted into the data set.  Also, if signals are below 
    a threshold, the threshold value is inserted.
    PARAMETERS
    ----------
    df 
        DESCRIPTION The Pandas dataframe containing the values to be analyzed or 
        plotted
    vmin The minimum value of the data entries in each cell
        DESCRIPTION The minimum value of the data entries in each cell. Default 
        value = 1.e-3
        TYPE Floating point
    excess_time_factor 
        DESCRIPTION multiplier used to set boundaries on scan time durations.
        Default value = 1.1
        TYPE Floating point
    scantime
        DESCRIPTION The number of seconds for a scan.  Default value = 150 s.
        TYPE Float
    RETURNS
    df_cleaned, 
        DESCRIPTION The cleaned pandas data frame containing the entire 
        dataset after cleaning
        TYPE Pandas dataframe with timezone naive datetime index and measured values
    '''
    
    # Ensure that the 'Datetime' column is a datetime object before processing
    dt = pd.to_datetime(df.index)
    ndt = len(dt)
    tdiffs = (dt[1:] - dt[0:-1]) / 1.e9    # Time differences in seconds
    tdiffsint = np.int64(tdiffs)
    ####### Chat GPT helped make the following code to insert NaN values into the DataFrame where there are gaps in the data
    
    
    # Create an empty list to collect rows that will be inserted
    new_rows = []

    # Loop through all intervals between recorded scans
    for i in range(ndt - 1):  
        # If the gap between two scans exceeds the threshold, insert NaN values
        if tdiffsint[i] > scantime * (1 + excess_time_factor):
            num_missing = int(tdiffsint[i] // scantime) - 1  # Number of missing scans
            
            # Create new timestamps and insert rows with NaN values
            for j in range(num_missing):
                time_of_new_scan = pd.to_datetime(df.index[i]) + pd.Timedelta(seconds=(scantime * (j + 1)))
                new_row = pd.Series([np.nan] * len(df.columns), index=df.columns, name=time_of_new_scan)
                new_rows.append(new_row)
    
    # Create a DataFrame from the rows that need to be inserted
    dfadd = pd.DataFrame(new_rows)

    # Concatenate the original DataFrame and the new rows
    df_cleaned = pd.concat([df, dfadd])

    # Ensure the index is still datetime after concatenation
    df_cleaned.index = pd.to_datetime(df_cleaned.index)

    # Sort the DataFrame to ensure the rows are in order (chronological order)
    df_cleaned = df_cleaned.sort_index()

    # Convert all columns to numeric values (ignoring errors)
    df_cleaned = df_cleaned.apply(pd.to_numeric, errors='coerce')

    # Clip the data to set values below `vmin` to `vmin`
    df_cleaned = df_cleaned.clip(lower=vmin)
    
    return df_cleaned


# Run the functions and create new dataframe with the filled time gaps:

In [66]:
dfraw = loadSMPSdata()
# print ('________________  raw data  ___________________\n\n', dfraw, '\n\n')
df = clean_data( dfraw )
# print ( ' ________________ clipped data ___________\n', df, '\n' )

# Add the datetime column back:

In [67]:
# Add the new 'DateTime Sample Start' column
df.insert(0, 'DateTime Sample Start', df.index)

# Now 'DateTime Sample Start' is the first column, with values from the index


In [68]:
df

,DateTime Sample Start,Scan Number,Test Name,Detector Status,Classifier Errors,Communication Status,Neutralizer Status,Detector Inlet Flow (L/min),Detector Counting Flow (L/min),Impactor Flow (L/min),...,572.55,593.52,615.27,637.80,661.17,685.39,710.50,736.53,763.51,791.48
2025-04-24 00:01:13,2025-04-24 00:01:13,574.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,0.010,0.010,50.745,0.010,52.524,0.010,0.010,0.010,38.041,31.999
2025-04-24 00:03:45,2025-04-24 00:03:45,575.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,49.424,0.010,51.130,0.010,0.010,53.857,0.010,48.486,0.010,0.010
2025-04-24 00:06:16,2025-04-24 00:06:16,576.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,48.989,0.010,0.010,0.010,52.524,53.415,0.010,0.010,0.010,0.010
2025-04-24 00:08:47,2025-04-24 00:08:47,577.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,49.424,0.010,0.010,0.010,0.010,53.415,54.301,0.010,0.010,0.010
2025-04-24 00:11:18,2025-04-24 00:11:18,578.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,48.989,49.863,0.010,52.075,0.010,53.415,0.010,48.319,38.500,0.010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-15 03:52:24,2025-08-15 03:52:24,4673.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,49.449,0.010,0.010,51.662,0.010,53.445,0.010,0.010,0.010,59.839
2025-08-15 03:54:55,2025-08-15 03:54:55,4674.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,0.010,0.010,101.543,0.010,52.553,0.010,54.328,0.010,0.010,26.347
2025-08-15 03:57:26,2025-08-15 03:57:26,4675.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,98.464,100.220,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010
2025-08-15 03:59:58,2025-08-15 03:59:58,4676.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,0.010,49.890,0.010,0.010,0.010,53.883,57.489,0.010,0.010,0.010


## Change the local timezone below to your local timezone

- This function "convert_to_local" takes the DateTime Sample Start column which is in UTC and converts it to a column titled "local_time"

- You will need to specify your specific timezone in this line: local_timezone = pytz.timezone('America/Los_Angeles')

In [69]:
def convert_to_local(df):

    df_local = df.copy()

    # Local timezone (replace with your local timezone)
    source_timezone = pytz.timezone('UTC')
    local_timezone = pytz.timezone('America/Los_Angeles') # <---- change timezone here to your own if needed!!!

    # Use the tz_convert function to convert the datetime column
    df_local['DateTime Sample Start'] = df['DateTime Sample Start'].dt.tz_localize(source_timezone).dt.tz_convert(local_timezone)
    df_local['DateTime Sample Start'] = pd.to_datetime(df_local["DateTime Sample Start"].dt.strftime('%Y-%m-%d %H:%M:%S'))

    df_local.rename(columns={'DateTime Sample Start':'local_time'}, inplace=True)
    return df_local

In [70]:
df_local = convert_to_local(df)

In [71]:
df_local # here you can check to make sure the time column has properly changed from UTC to your local time 

,local_time,Scan Number,Test Name,Detector Status,Classifier Errors,Communication Status,Neutralizer Status,Detector Inlet Flow (L/min),Detector Counting Flow (L/min),Impactor Flow (L/min),...,572.55,593.52,615.27,637.80,661.17,685.39,710.50,736.53,763.51,791.48
2025-04-24 00:01:13,2025-04-23 17:01:13,574.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,0.010,0.010,50.745,0.010,52.524,0.010,0.010,0.010,38.041,31.999
2025-04-24 00:03:45,2025-04-23 17:03:45,575.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,49.424,0.010,51.130,0.010,0.010,53.857,0.010,48.486,0.010,0.010
2025-04-24 00:06:16,2025-04-23 17:06:16,576.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,48.989,0.010,0.010,0.010,52.524,53.415,0.010,0.010,0.010,0.010
2025-04-24 00:08:47,2025-04-23 17:08:47,577.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,49.424,0.010,0.010,0.010,0.010,53.415,54.301,0.010,0.010,0.010
2025-04-24 00:11:18,2025-04-23 17:11:18,578.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,48.989,49.863,0.010,52.075,0.010,53.415,0.010,48.319,38.500,0.010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-15 03:52:24,2025-08-14 20:52:24,4673.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,49.449,0.010,0.010,51.662,0.010,53.445,0.010,0.010,0.010,59.839
2025-08-15 03:54:55,2025-08-14 20:54:55,4674.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,0.010,0.010,101.543,0.010,52.553,0.010,54.328,0.010,0.010,26.347
2025-08-15 03:57:26,2025-08-14 20:57:26,4675.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,98.464,100.220,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010
2025-08-15 03:59:58,2025-08-14 20:59:58,4676.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,0.010,49.890,0.010,0.010,0.010,53.883,57.489,0.010,0.010,0.010


In [74]:
df_local

,local_time,Scan Number,Test Name,Detector Status,Classifier Errors,Communication Status,Neutralizer Status,Detector Inlet Flow (L/min),Detector Counting Flow (L/min),Impactor Flow (L/min),...,572.55,593.52,615.27,637.80,661.17,685.39,710.50,736.53,763.51,791.48
2025-04-24 00:01:13,2025-04-23 17:01:13,574.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,0.010,0.010,50.745,0.010,52.524,0.010,0.010,0.010,38.041,31.999
2025-04-24 00:03:45,2025-04-23 17:03:45,575.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,49.424,0.010,51.130,0.010,0.010,53.857,0.010,48.486,0.010,0.010
2025-04-24 00:06:16,2025-04-23 17:06:16,576.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,48.989,0.010,0.010,0.010,52.524,53.415,0.010,0.010,0.010,0.010
2025-04-24 00:08:47,2025-04-23 17:08:47,577.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,49.424,0.010,0.010,0.010,0.010,53.415,54.301,0.010,0.010,0.010
2025-04-24 00:11:18,2025-04-23 17:11:18,578.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,48.989,49.863,0.010,52.075,0.010,53.415,0.010,48.319,38.500,0.010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-15 03:52:24,2025-08-14 20:52:24,4673.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,49.449,0.010,0.010,51.662,0.010,53.445,0.010,0.010,0.010,59.839
2025-08-15 03:54:55,2025-08-14 20:54:55,4674.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,0.010,0.010,101.543,0.010,52.553,0.010,54.328,0.010,0.010,26.347
2025-08-15 03:57:26,2025-08-14 20:57:26,4675.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,98.464,100.220,0.010,0.010,0.010,0.010,0.010,0.010,0.010,0.010
2025-08-15 03:59:58,2025-08-14 20:59:58,4676.0,NaN,NaN,NaN,0.01,NaN,0.6,0.3,0.6,...,0.010,49.890,0.010,0.010,0.010,53.883,57.489,0.010,0.010,0.010


# Save this cleaned CSV file and you are ready to go! 

In [75]:
# Save DataFrame to a CSV file in a specific plot_ready_folder (absolute path)

df_local.to_csv(f'{plot_ready_folder}/{cleaned_csv}', index=False)  # `index=False` to not include the index